In [1]:
import os
from PIL import Image
import numpy as np

path = "/home/karkin.mun/training_images/"

import tensorflow as tf


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17591466470511647850
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6829089025139680325
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16544995741335878001
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10909243802
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6189149776246720501
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
]


In [4]:
valid_images = [".jpg",".gif",".png",".tga"]

imgs = []
labels = [] 

size = (300, 300)

for folders in os.listdir(path):
    
    if folders == 'PROHIBITED':
        continue 
    
    
    new_path = path + folders
    for f in os.listdir(new_path):
        
        ext = os.path.splitext(f)[1]
        
        if ext.lower() not in valid_images:
            continue
            
        im = Image.open(os.path.join(new_path,f)).convert("RGB")
        width, height = im.size
        
        if width < 300 or height < 300:
            continue 
            
        im = im.resize((size[0], size[1]), Image.ANTIALIAS)
        
        imgs.append(np.array(im).reshape(300, 300, 3)) # .convert("L")
        labels.append(folders)
                    

/home/karkin.mun/environments/ml_dir/lib/python3.5/site-packages/PIL/Image.py:966: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [5]:
len(imgs), len(labels)

(3203, 3203)

In [6]:
# imgs[0].size
imgs[0].shape

(300, 300, 3)

In [7]:
d = {}

for im in imgs:
    if im.shape[:2] not in d:
        d[im.shape[:2]] = 0 
    
    d[im.shape[:2]] += 1  

In [8]:
sorted(d.items(), key = lambda x: x[1], reverse = True)

[((300, 300), 3203)]

In [9]:
label_dict = {'ADULT':0, 'COUNTERFEIT':1, 'LEGIT':2, 'PHARMA':3, 'SMOKE':4,'TMS':5, 'WEAPON':6}

for i in range(len(labels)):
    labels[i] = label_dict[labels[i]]

In [10]:
from keras.utils import to_categorical

labels_ohc = to_categorical(labels)

labels_ohc[:5]

imgs = np.array(imgs).astype('float32') / 255.

In [11]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(np.array(imgs), labels_ohc, test_size=0.2, stratify = labels_ohc)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2562, 300, 300, 3), (641, 300, 300, 3), (2562, 7), (641, 7))

In [13]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

"""
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=20, 
                             zoom_range=0.15,
                             width_shift_range=0.2, 
                             height_shift_range=0.2, 
                             shear_range=0.15,
                             horizontal_flip=True, 
                             fill_mode="nearest")
"""
pass

In [14]:
import keras
import numpy as np
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
 
#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(300,300,3))
 
#Load the Inception_V3 model
#inception_model = inception_v3.InceptionV3(weights='imagenet')
 
#Load the ResNet50 model
#resnet_model = resnet50.ResNet50(weights='imagenet')
 
#Load the MobileNet model
#mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [15]:

import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50

model_aug=Sequential()
model_aug.add(vgg_model)

top_model=Sequential()
top_model.add(Flatten(input_shape=(9, 9, 512)))
#model_aug.add(Dropout(0.3))
top_model.add(Dense(100, activation='relu'))

top_model.add(Dense(7, activation='sigmoid'))

model_aug.add(top_model)

for layer in model_aug.layers[0].layers[:17]:
    layer.trainable=False
    
model_aug.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-6), metrics=['accuracy'])

In [16]:
model_aug.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
sequential_2 (Sequential)    (None, 7)                 4148007   
Total params: 18,862,695
Trainable params: 6,507,815
Non-trainable params: 12,354,880
_________________________________________________________________


In [17]:
model_aug.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
__________

In [18]:
model_aug.layers[1].summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               4147300   
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 707       
Total params: 4,148,007
Trainable params: 4,148,007
Non-trainable params: 0
_________________________________________________________________


In [19]:
"""
H = model_aug.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                        validation_data=(x_test, x_test), steps_per_epoch=len(x_train) // batch_size,
                        epochs=50)
"""

from keras.callbacks import EarlyStopping
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint

csv_logger = CSVLogger('log.csv', append=True, separator=';')

es = EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=5)

checkpoint = ModelCheckpoint('OSM_vgg16_imgs',
                            monitor='acc',
                            verbose=5, # at the end of every 5th epoch 
                            save_best_only=True,
                            mode='max')

#history = model_aug.fit(x_train, y_train, epochs = 10, verbose = 1, validation_data = (x_test, y_test), callbacks=[es, csv_logger]) #)

In [23]:
with tf.device('/cpu:0'):
    model_aug.predict_classes(x_test[:2])

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node vgg16/block1_conv1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](vgg16/block1_conv1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, block1_conv1/kernel/read)]]
	 [[{{node sequential_2/dense_2/Sigmoid/_181}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_166_sequential_2/dense_2/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]